In [156]:
#Harrison Ressler

#CAUTION: ONLY RUN SOME CELLS ONCE, ELSE IT WILL REDO THINGS BASED OFF OF PREVIOUS CHANGES ALREADY DONE
import pandas as pd

In [157]:
df = pd.read_csv("C:\\Users\\harri\\Desktop\\Cab_Project\\archive\\cab_rides.csv")
df.head()

,distance,cab_type,time_stamp,destination,source,price,surge_multiplier,id,product_id,name
0,0.44,Lyft,1544952607890,North Station,Haymarket Square,5.0,1.0,424553bb-7174-41ea-aeb4-fe06d4f4b9d7,lyft_line,Shared
1,0.44,Lyft,1543284023677,North Station,Haymarket Square,11.0,1.0,4bd23055-6827-41c6-b23b-3c491f24e74d,lyft_premier,Lux
2,0.44,Lyft,1543366822198,North Station,Haymarket Square,7.0,1.0,981a3613-77af-4620-a42a-0c0866077d1e,lyft,Lyft
3,0.44,Lyft,1543553582749,North Station,Haymarket Square,26.0,1.0,c2d88af2-d278-4bfd-a8d0-29ca77cc5512,lyft_luxsuv,Lux Black XL
4,0.44,Lyft,1543463360223,North Station,Haymarket Square,9.0,1.0,e0126e1f-8ca9-4f2e-82b3-50505a09db9a,lyft_plus,Lyft XL


In [158]:
data = df.copy() #Copying so I edit copy here and not core data
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 693071 entries, 0 to 693070
Data columns (total 10 columns):
distance            693071 non-null float64
cab_type            693071 non-null object
time_stamp          693071 non-null int64
destination         693071 non-null object
source              693071 non-null object
price               637976 non-null float64
surge_multiplier    693071 non-null float64
id                  693071 non-null object
product_id          693071 non-null object
name                693071 non-null object
dtypes: float64(3), int64(1), object(6)
memory usage: 52.9+ MB


In [159]:
##Clean Data
#Step 1: Remove blank spaces or nulls, and things I won't need.

#Drop id
data = data.drop(columns = "id")

In [160]:
#Remove Null rows : So, if any row has a null or NA it drops the ENTIRE row
data = data.dropna()

In [161]:
data.info() #Seeing how many rows dropped.
#Previous: 693,071
#Current: 637,976

<class 'pandas.core.frame.DataFrame'>
Int64Index: 637976 entries, 0 to 693070
Data columns (total 9 columns):
distance            637976 non-null float64
cab_type            637976 non-null object
time_stamp          637976 non-null int64
destination         637976 non-null object
source              637976 non-null object
price               637976 non-null float64
surge_multiplier    637976 non-null float64
product_id          637976 non-null object
name                637976 non-null object
dtypes: float64(3), int64(1), object(5)
memory usage: 48.7+ MB


In [162]:
#Step 2: Grab Columns and clean up

data.describe()
#distance has a min of 0.02, max of 7.86 miles (presumably miles)
#surge_multiplier has a max of 3.0. Mean is 1.013, so presumably it doesn't happen much. Meaning its very constant, so it might not be a good variable.
#price generally from $7 to $25 (mean - std_dev = $7, mean + std_dev = $25)
#time_stamp ... I'll need ot convert this into something else. So, currently its useless.

,distance,time_stamp,price,surge_multiplier
count,637976.000000,6.379760e+05,637976.000000,637976.000000
mean,2.189261,1.544046e+12,16.545125,1.015068
std,1.135413,6.892028e+08,9.324359,0.095422
min,0.020000,1.543204e+12,2.500000,1.000000
25%,1.270000,1.543444e+12,9.000000,1.000000
50%,2.160000,1.543737e+12,13.500000,1.000000
75%,2.930000,1.544828e+12,22.500000,1.000000
max,7.860000,1.545161e+12,97.500000,3.000000


In [163]:
#Convert Columns from info above into something useful

#I should probably make a correlation matrix here.

data.corr()
#Distance, price, & surge_multiplier seem fine.

,distance,time_stamp,price,surge_multiplier
distance,1.000000,0.003290,0.345061,0.025946
time_stamp,0.003290,1.000000,0.000808,0.000611
price,0.345061,0.000808,1.000000,0.240458
surge_multiplier,0.025946,0.000611,0.240458,1.000000


In [164]:
#Now encode categorical (non-numeric) data

len(data.product_id.factorize()[0]) # [0] shows first array instead of index, & len() shows its length
data["product_id"], product_id_labels = data.product_id.factorize() #Converting strings, into numbers that I can use (Ex: lyft_line -> 0, lyft_priemer -> 1, etc.)
#product_id_labels allows me to link back to what the string is. 
#GOOGLE HOW TO DO ABOVE: "GOOGLE: REVERSE PANDAS FACTORIZE"
print(data[:10])
#Shows that I put product_id it puts into numbers

   distance cab_type     time_stamp              destination  \
0      0.44     Lyft  1544952607890            North Station   
1      0.44     Lyft  1543284023677            North Station   
2      0.44     Lyft  1543366822198            North Station   
3      0.44     Lyft  1543553582749            North Station   
4      0.44     Lyft  1543463360223            North Station   
5      0.44     Lyft  1545071112138            North Station   
6      1.08     Lyft  1543208580200  Northeastern University   
7      1.08     Lyft  1543780384677  Northeastern University   
8      1.08     Lyft  1543818482645  Northeastern University   
9      1.08     Lyft  1543315522249  Northeastern University   

             source  price  surge_multiplier  product_id          name  
0  Haymarket Square    5.0               1.0           0        Shared  
1  Haymarket Square   11.0               1.0           1           Lux  
2  Haymarket Square    7.0               1.0           2          Lyft  
3  

In [165]:
print(product_id_labels) #Why are there numbers?
#Should see the number ones are a significant percent of the data, els, filter out.

Index(['lyft_line', 'lyft_premier', 'lyft', 'lyft_luxsuv', 'lyft_plus',
       'lyft_lux', '6f72dfc5-27f1-42e8-84db-ccc7a75f6969',
       '6c84fd89-3f11-4782-9b50-97c468b19529',
       '55c66225-fbe7-4fd5-9072-eab1ece5e23e',
       '9a0e7b09-b92b-4c41-9779-2ad22b4d779d',
       '6d318bcc-22a3-4af6-bddd-b409bfce1546',
       '997acbb5-e102-41e1-b155-9df7de0a73f2'],
      dtype='object')


In [166]:
#Shows how much of each each value (Ex: lyft-line) there is.
df['product_id'].value_counts()
#Wow. There is 55096 of stuff that I don't know what it is. That is a significant protion of the data.
#I'm guessing the people who put the data togtehr did it intentionally.
#But what is it? Random taxi cab companies?

#LOOK AT THE SITE YOU GOT IT FROM LATER (KAGGLE)

9a0e7b09-b92b-4c41-9779-2ad22b4d779d    55096
6f72dfc5-27f1-42e8-84db-ccc7a75f6969    55096
6d318bcc-22a3-4af6-bddd-b409bfce1546    55096
6c84fd89-3f11-4782-9b50-97c468b19529    55095
8cf7e821-f0d3-49c6-8eba-e679c0ebcf6a    55095
55c66225-fbe7-4fd5-9072-eab1ece5e23e    55094
997acbb5-e102-41e1-b155-9df7de0a73f2    55091
lyft_premier                            51235
lyft_lux                                51235
lyft_luxsuv                             51235
lyft                                    51235
lyft_plus                               51235
lyft_line                               51233
Name: product_id, dtype: int64

In [167]:
print(data[:10])

   distance cab_type     time_stamp              destination  \
0      0.44     Lyft  1544952607890            North Station   
1      0.44     Lyft  1543284023677            North Station   
2      0.44     Lyft  1543366822198            North Station   
3      0.44     Lyft  1543553582749            North Station   
4      0.44     Lyft  1543463360223            North Station   
5      0.44     Lyft  1545071112138            North Station   
6      1.08     Lyft  1543208580200  Northeastern University   
7      1.08     Lyft  1543780384677  Northeastern University   
8      1.08     Lyft  1543818482645  Northeastern University   
9      1.08     Lyft  1543315522249  Northeastern University   

             source  price  surge_multiplier  product_id          name  
0  Haymarket Square    5.0               1.0           0        Shared  
1  Haymarket Square   11.0               1.0           1           Lux  
2  Haymarket Square    7.0               1.0           2          Lyft  
3  

In [168]:
#Do same as above for factorizeing data with other non-numerical data (cab_type & name)
data["cab_type"], cab_type_labels = data.cab_type.factorize() #Converting strings, into numbers that I can use (Ex: Lyft -> 0, uber -> 1, etc.)
#cab_type_labels allows me to link back to what the string is. 

In [169]:
data["name"], name_labels = data.name.factorize() #Converting strings, into numbers that I can use (Ex: shared -> 0, Lux -> 1, etc.)
#name_labels allows me to link back to what the string is. 

In [170]:
print(data[:10])

   distance  cab_type     time_stamp              destination  \
0      0.44         0  1544952607890            North Station   
1      0.44         0  1543284023677            North Station   
2      0.44         0  1543366822198            North Station   
3      0.44         0  1543553582749            North Station   
4      0.44         0  1543463360223            North Station   
5      0.44         0  1545071112138            North Station   
6      1.08         0  1543208580200  Northeastern University   
7      1.08         0  1543780384677  Northeastern University   
8      1.08         0  1543818482645  Northeastern University   
9      1.08         0  1543315522249  Northeastern University   

             source  price  surge_multiplier  product_id  name  
0  Haymarket Square    5.0               1.0           0     0  
1  Haymarket Square   11.0               1.0           1     1  
2  Haymarket Square    7.0               1.0           2     2  
3  Haymarket Square   26

In [171]:
print(df[:10]) #Showing original data un-cleaned

   distance cab_type     time_stamp              destination  \
0      0.44     Lyft  1544952607890            North Station   
1      0.44     Lyft  1543284023677            North Station   
2      0.44     Lyft  1543366822198            North Station   
3      0.44     Lyft  1543553582749            North Station   
4      0.44     Lyft  1543463360223            North Station   
5      0.44     Lyft  1545071112138            North Station   
6      1.08     Lyft  1543208580200  Northeastern University   
7      1.08     Lyft  1543780384677  Northeastern University   
8      1.08     Lyft  1543818482645  Northeastern University   
9      1.08     Lyft  1543315522249  Northeastern University   

             source  price  surge_multiplier  \
0  Haymarket Square    5.0               1.0   
1  Haymarket Square   11.0               1.0   
2  Haymarket Square    7.0               1.0   
3  Haymarket Square   26.0               1.0   
4  Haymarket Square    9.0               1.0   
5  Haym

In [172]:
#source & destination are unique because both should be lined up numerically with each other
# because a person might be going from place A to B. Or vice vers (B to A)

#concatonate to a pandas columns

c = pd.concat([data.source, data.destination])
print(c)

0         Haymarket Square
1         Haymarket Square
2         Haymarket Square
3         Haymarket Square
4         Haymarket Square
                ...       
693065           North End
693066           North End
693067           North End
693069           North End
693070           North End
Length: 1275952, dtype: object


In [173]:
#Now factorize it like others
c_fact , c_labels = c.factorize() #Converting strings, into numbers that I can use (Ex: Haymarket Square -> 0, Back Bay -> 1, etc.)
print(c_fact, c_labels)

[0 0 0 ... 2 2 2] Index(['Haymarket Square', 'Back Bay', 'North End', 'North Station',
       'Beacon Hill', 'Boston University', 'Fenway', 'South Station',
       'Theatre District', 'West End', 'Financial District',
       'Northeastern University'],
      dtype='object')


In [174]:
#Now split it up, and put back into df coulmns(source & destination)
c_half1 = c_fact[:len(c_fact)//2] #Grabs all of the stuff from begging to halfway through. "//2" divides by 2 & returns an interger since it rounds

In [175]:
print(len(c_half1)) #63k is the first half

637976


In [176]:
c_half2 = c_fact[len(c_fact)//2:] #Grabs all of the stuff from 2nd half. "//2" divides by 2 & returns an interger since it rounds

In [177]:
print(len(c_half2))
#Matches length of c_half1, so its good, & we aren't missing any.

637976


In [178]:
data["source"] = c_half1 #Since c_half1 is source
data["destination"] = c_half2 #Since c_half2 is destination

In [179]:
print(data) #Checking it worked. Yes, it matches the numers given above.

        distance  cab_type     time_stamp  destination  source  price  \
0           0.44         0  1544952607890            3       0    5.0   
1           0.44         0  1543284023677            3       0   11.0   
2           0.44         0  1543366822198            3       0    7.0   
3           0.44         0  1543553582749            3       0   26.0   
4           0.44         0  1543463360223            3       0    9.0   
...          ...       ...            ...          ...     ...    ...   
693065      1.00         1  1543708385534            2       9    9.5   
693066      1.00         1  1543708385534            2       9   13.0   
693067      1.00         1  1543708385534            2       9    9.5   
693069      1.00         1  1543708385534            2       9   27.0   
693070      1.00         1  1543708385534            2       9   10.0   

        surge_multiplier  product_id  name  
0                    1.0           0     0  
1                    1.0         

In [180]:
#Now we should fix time_stamp

#Converts time after 1970:
data["time_stamp"] = pd.to_datetime(data.time_stamp, unit = "ms")
#Since data has 13 numbers, unit is "Millseconds", default for paramter is seconds, when tried default said first day was 1970, uber not in 1970
#It says it starts in 2018. That sounds probably correct
#data = ... saves it to data

In [181]:
print(data) #Checking if time_stamp useable

        distance  cab_type              time_stamp  destination  source  \
0           0.44         0 2018-12-16 09:30:07.890            3       0   
1           0.44         0 2018-11-27 02:00:23.677            3       0   
2           0.44         0 2018-11-28 01:00:22.198            3       0   
3           0.44         0 2018-11-30 04:53:02.749            3       0   
4           0.44         0 2018-11-29 03:49:20.223            3       0   
...          ...       ...                     ...          ...     ...   
693065      1.00         1 2018-12-01 23:53:05.534            2       9   
693066      1.00         1 2018-12-01 23:53:05.534            2       9   
693067      1.00         1 2018-12-01 23:53:05.534            2       9   
693069      1.00         1 2018-12-01 23:53:05.534            2       9   
693070      1.00         1 2018-12-01 23:53:05.534            2       9   

        price  surge_multiplier  product_id  name  
0         5.0               1.0           0    

In [182]:
#Now we will get the numbers (Ex: Year, Day, Hour etc) that is likely useful in predicting pay
#Google: "Pandas Hours from datetime" -> once we find out how to do one (Ex: Hour, or day, or year) we can do the other.

data["minute"] = data.time_stamp.dt.minute
data["hour"] = data.time_stamp.dt.hour #making new column "hour". dt = datetime
data["day"] = data.time_stamp.dt.day
data["month"] = data.time_stamp.dt.month
data["year"] = data.time_stamp.dt.year

In [183]:
print(data) #Checking that numbers match (Ex: time_stamp : 2018 ..., year: 2018) and do for all new columns

        distance  cab_type              time_stamp  destination  source  \
0           0.44         0 2018-12-16 09:30:07.890            3       0   
1           0.44         0 2018-11-27 02:00:23.677            3       0   
2           0.44         0 2018-11-28 01:00:22.198            3       0   
3           0.44         0 2018-11-30 04:53:02.749            3       0   
4           0.44         0 2018-11-29 03:49:20.223            3       0   
...          ...       ...                     ...          ...     ...   
693065      1.00         1 2018-12-01 23:53:05.534            2       9   
693066      1.00         1 2018-12-01 23:53:05.534            2       9   
693067      1.00         1 2018-12-01 23:53:05.534            2       9   
693069      1.00         1 2018-12-01 23:53:05.534            2       9   
693070      1.00         1 2018-12-01 23:53:05.534            2       9   

        price  surge_multiplier  product_id  name  minute  hour  day  month  \
0         5.0       

In [184]:
#It looks like all the data is now numeric, useable, and clean.

##Phase 2: Now lets begin working on Model Selection

#So, to use the models, we need to split the data up into training data, & testing data.

from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(data, test_size=0.2, random_state=1) 
#test_size is % of data used for testing, random_state is used for testing where it allows a random genrator to create data from the number given -
#(Ex: If 1, then numbers will be generated around 1, can set consistancy by choosing where it randomly gerneates around (in this case 1))

In [185]:
print("TrainSet:\n", train_set.count()) #Can see that it is 80% of the data
print("\nTestSet:\n", test_set.count()) #Can see that it is 20% of the data

TrainSet:
 distance            510380
cab_type            510380
time_stamp          510380
destination         510380
source              510380
price               510380
surge_multiplier    510380
product_id          510380
name                510380
minute              510380
hour                510380
day                 510380
month               510380
year                510380
dtype: int64

TestSet:
 distance            127596
cab_type            127596
time_stamp          127596
destination         127596
source              127596
price               127596
surge_multiplier    127596
product_id          127596
name                127596
minute              127596
hour                127596
day                 127596
month               127596
year                127596
dtype: int64


In [186]:
#Now that its split, we can start trying various models.

#Separate Predictors (X variables) and target/outcome (y variable)
dropped_variables = ["price", "time_stamp", "destination"] #List of variables that will not be used as X variables

#Training will be used to train the data(80% of data). Test will be used ot compare the data to TEST if its correct (20% of the data)
X_train = train_set.drop(dropped_variables, axis=1) #X variables are trying to predict Y. Don't give it Y. (Y is price) And time_stamp would not be helpful (But Hour, month etc would), so drop
y_train = train_set["price"].copy() #price is the y variable
X_test = test_set.drop(dropped_variables, axis=1)
y_test = test_set["price"].copy()

In [187]:
X_train.info() #Data looks ok. Nothings there that shouldn't be there.
#So, its probably true for other newly created stuff from above (y_train, X_test, y_test)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 510380 entries, 613406 to 139061
Data columns (total 11 columns):
distance            510380 non-null float64
cab_type            510380 non-null int64
source              510380 non-null int64
surge_multiplier    510380 non-null float64
product_id          510380 non-null int64
name                510380 non-null int64
minute              510380 non-null int64
hour                510380 non-null int64
day                 510380 non-null int64
month               510380 non-null int64
year                510380 non-null int64
dtypes: float64(2), int64(9)
memory usage: 46.7 MB


In [188]:
###############"Visualize the data to gain insights"
#Month to anything
#Determine pay per hour - ASK NAJAFI

In [189]:
##########"LOOK FOR CORRELATIONS"

In [190]:
####THINK ABOUT EXPERIMENT WITH ATTRIBUTE COMBINATION

In [191]:
####"Separate the Predictors and the Labels" if models need it

In [192]:
#NOTE TO SELF: CURRENTLY I AM JUST TRYING AS MANY THINGS AS I CAN AND HIOPING ONE STICKS.
#I DON'T THINK THATS A GOOD SYSTEM. FIND A DEDICATED PROCESS TO DETERRMINE WHICH MODEL

#Linear Regression
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
linear_reg = LinearRegression() #The actual LinearRegression
#Cross validation says how well it fits (its kind of like the R^2)
scores = cross_val_score(linear_reg, X_train, y_train, scoring="neg_mean_squared_error", cv=10) 
#cv means split it into 90% of data & tests on the 10% of data, & does it 10 times, 1 time for each of the 10%'s of the data. So all the data gets tested

In [193]:
print("Negative MSE:\n", scores)

#INTRPRET DATA

Negative MSE:
 [-66.85353755 -67.73789277 -67.38375765 -67.94718194 -67.15123786
 -67.15578224 -67.28593613 -67.50570805 -66.44669993 -67.10832129]


In [194]:
import numpy as np
linear_rmse_scores = np.sqrt(-scores)
print("\nRMSE:\n", linear_rmse_scores)
#Says we are off on averge about $8.2 (plus or minus)


RMSE:
 [8.17640126 8.23030332 8.20876103 8.24300806 8.19458589 8.19486316
 8.20280051 8.21618574 8.15148452 8.19196688]


In [195]:
average_cross_validation_Linear_RMSE = linear_rmse_scores.mean();
print("\nAverange Cross-Validation RMSE:", average_cross_validation_Linear_RMSE)

#Here is the avg. of about how off we are from the target/Y 
#So on avg we are off by $8.2


Averange Cross-Validation RMSE: 8.201036036366057


In [196]:
target_range = y_train.max()-y_train.min();
print("\nTarget Range: (", y_train.min(), ", ", y_train.max(), ") ==> ", target_range)
print("\nRMSE relative to range: {0:0.2%}".format(average_cross_validation_Linear_RMSE/target_range))
#So, if the max is $97.5 and the min is $2.5, and we are off by 8.63%. So, not sucha big deal in big trasnactions. But could be BAD in small ones.


Target Range: ( 2.5 ,  97.5 ) ==>  95.0

RMSE relative to range: 8.63%


In [197]:
#Ok, so now lets try anotherr model & see how we do.

#Copy & Paste Above Linear Regression, just replace one line

#2nd Degree Polynomial:
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
from sklearn.preprocessing import PolynomialFeatures

#Poly..Features: Squares and multiplies to make new features/columns/X-variables
poly_features = PolynomialFeatures(degree=2, include_bias=False)
X_train_poly = poly_features.fit_transform(X_train)

#Runs Regressions
linear_reg = LinearRegression()
ridge_reg = Ridge(alpha=0.1, solver="cholesky", max_iter=1000)
lasso_reg = Lasso(alpha=0.1, max_iter=1000)
elastic_net_reg = ElasticNet(alpha=0.1, l1_ratio=0.5, max_iter=1000)
tree_reg = DecisionTreeRegressor()
forest_reg = RandomForestRegressor()

In [ ]:
#Takes time! (Like 15 minutes plus)

#Disregard any warnings. IT IS STILL RUNNING!!!!!!

print("training the linear model")
linear_scores = cross_val_score(linear_reg, X_train, y_train, scoring="neg_mean_squared_error", cv=10)
print("done training the linear model -> starting the poly model")
poly_scores = cross_val_score(linear_reg, X_train_poly, y_train, scoring="neg_mean_squared_error", cv=10)
print("done training the poly model -> starting ridge model")
ridge_scores = cross_val_score(linear_reg, X_train_poly, y_train, scoring="neg_mean_squared_error", cv=10)
print("done training the ridge model -> starting lasso model")
lasso_scores = cross_val_score(lasso_reg, X_train_poly, y_train, scoring="neg_mean_squared_error", cv=10)
print("done training the lasso model -> startiong elastic net model")
elastic_net_scores = cross_val_score(elastic_net_reg, X_train_poly, y_train, scoring="neg_mean_squared_error", cv=10)
print("done training the elastic net model -> starting decission tree model")
tree_scores = cross_val_score(tree_reg, X_train, y_train, scoring="neg_mean_squared_error", cv=10)
print("done training the decission tree model -> starting random forest model")
forest_scores = cross_val_score(forest_reg, X_train, y_train, scoring="neg_mean_squared_error", cv=10)
print("done training the random forest tree model\n")

In [44]:
#Calculations for graph below
target_range = y_train.max()-y_train.min();
linear_average_RMSE = np.sqrt(-linear_scores).mean()/target_range;
poly_average_RMSE = np.sqrt(-poly_scores).mean()/target_range;
ridge_average_RMSE = np.sqrt(-ridge_scores).mean()/target_range;
lasso_average_RMSE = np.sqrt(-lasso_scores).mean()/target_range;
elastic_net_average_RMSE = np.sqrt(-elastic_net_scores).mean()/target_range;
tree_average_RMSE = np.sqrt(-tree_scores).mean()/target_range;
forest_average_RMSE = np.sqrt(-forest_scores).mean()/target_range;

NameError: name 'lasso_scores' is not defined

In [45]:
x = [1, 2, 3, 4, 5, 6, 7]
model_names = ["linear", "Poly", "ridge", "lasso", "elastic net", "decision tree", "random forest"]
average_RMSEs = np.array([linear_average_RMSE, poly_average_RMSE, ridge_average_RMSE, lasso_average_RMSE, elastic_net_average_RMSE, tree_average_RMSE, forest_average_RMSE])
print(['{:.2%}'.format(item) for item in average_RMSEs])
plt.figure(figsize=(10,5))
plt.bar(x, average_RMSEs)
plt.xticks(x, model_names)
plt.ylim(ymax=.1) #Max is .1 (= 10% error)
plt.show()

#decision tree & random forrest only are off 2.44% & 1.93% of the time. They are the best so far
#So, they are by about $2
#So, customers (cab drivers) I would tell them they would make -$2 on AVERAGE. FIND CASES WHEN ITS WORSE THAN THAT AND COMPENSATE

NameError: name 'lasso_average_RMSE' is not defined

In [ ]:
#Now tune the best models.
#Find out how to tune models...hyerparamters
from sklearn.model_selection import GridSearchCV
param_grid = [
{'bootstrap': [False], 'max_features': [2, 4, 6], 'n_estimators': [3, 10]},
{'bootstrap': [True, False], 'max_features': [2, 6], 'n_estimators': [3, 10]},
]
forest_reg = RandomForestRegressor()
grid_search = GridSearchCV(forest_reg, param_grid, cv=10,scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)
print(grid_search.best_params_)

In [198]:
#Insert stuff from above into hyperparameters

#Taking best reg & imporving with hyperparamters
new_forest_reg = RandomForestRegressor(bootstrap=True, max_features=6, n_estimators=10)

In [199]:
new_forest_reg.fit(X_train, y_train) #Pass in x,y data

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features=6, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=10,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [200]:
import joblib

In [201]:
#Save to file
joblib.dump(new_forest_reg, "saved_model") #Saves file from new_forest_reg, saves as "saved_model"

['saved_model']

In [202]:
saved_model = joblib.load("saved_model") #loads file

In [203]:
print(saved_model)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features=6, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=10,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)


In [4]:
import sklearn
print(sklearn.show_versions())


System:
    python: 3.7.4 (default, Aug  9 2019, 18:34:13) [MSC v.1915 64 bit (AMD64)]
executable: C:\Users\harri\Anaconda3\python.exe
   machine: Windows-10-10.0.19041-SP0

Python deps:
       pip: 19.2.3
setuptools: 41.4.0
   sklearn: 0.21.3
     numpy: 1.16.5
     scipy: 1.3.1
    Cython: 0.29.13
    pandas: 0.25.1
None
